<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex07/patrick_ferreira/ex07_patrick_ferreira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from asyncio import Queue
from threading import Thread

nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


#  Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural simples para prever a proxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Língua".

Este dataset já possui um tamanho razoável e é bem provável que você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!python -m pip install transformers

## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 18 21:30:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   55C    P0    N/A /  N/A |    230MiB /  2004MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from numpy.lib.format import open_memmap
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        # Escreva seu código aqui
        try:

            if len(texts) > 150:
                self.x = np.load("x_27675765.npy", mmap_mode="r", allow_pickle=True)
                self.y = np.load("y_27675765.npy", mmap_mode="r", allow_pickle=True)

                print("Carregando dataset preprocessado")

            else:
                print("Montando dataset")

                self.x = list()
                self.y = list()

                for text in texts:
                    tokens_key = tokenize(text, tokenizer)
                    for i in range(len(tokens_key)-context_size):
                        self.x.append(tokens_key[i:i+context_size])
                        self.y.append(tokens_key[i+context_size])

                self.x = np.array(self.x)
                self.y = np.array(self.y)

                np.save("x_" + str(self.x.shape[0]) + ".npy", self.x)
                np.save("y_" + str(self.y.shape[0]) + ".npy", self.y)

        except Exception as e:

            print("Excecao: ", e)

            print("Montando dataset")

            self.x = list()
            self.y = list()

            for text in texts:
                tokens_key = tokenize(text, tokenizer)
                for i in range(len(tokens_key)-context_size):
                    self.x.append(tokens_key[i:i+context_size])
                    self.y.append(tokens_key[i+context_size])

            self.x = np.array(self.x)
            self.y = np.array(self.y)

            np.save("x_" + str(self.x.shape[0]) + ".npy", self.x)
            np.save("y_" + str(self.y.shape[0]) + ".npy", self.y)

    def __len__(self):
        # Escreva seu código aqui
        return len(self.y)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        return torch.tensor(self.x[idx]).long(), torch.tensor(self.y[idx]).long()

## Teste se sua implementação do MyDataset está correta

In [7]:
# !jupyter kernelspec list

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Montando dataset
passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

File ‘sample_brwac.txt’ already there; not retrieving.



In [9]:
# Load datasets
context_size = 9

valid_examples = 101
test_examples = 100
texts = open('sample_brwac.txt', encoding="utf-8").readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Carregando dataset preprocessado
Montando dataset
Montando dataset


In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675765
valid examples: 82250
test examples: 166726


In [11]:
# Tool class for debug
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()

    def forward(self, x):
        # Do your print / debug stuff here
        print(x.shape)
        return x

In [12]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim, hidden_size):
        """
        Implements the Neural Language Model proposed by Bengio et al."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            hidden_size (int): Size of the hidden layer.
        """
        # Escreva seu código aqui.
        super().__init__()

        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)

        self.features = torch.nn.Sequential(
            nn.Linear(context_size * embedding_dim, 2 * hidden_size),
            # nn.LeakyReLU(),
            nn.Tanh(),
            # nn.Dropout(),
            nn.Linear(2 * hidden_size, hidden_size),
            # nn.LeakyReLU(),
            nn.Tanh(),
        )
        
        self.classifier = nn.Linear(hidden_size, vocab_size)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
        """
        # Escreva seu código aqui.
         
        return self.classifier(
            self.features(
                self.embedding(inputs).view(inputs.shape[0], -1)
            )
        )

## Teste o modelo com um exemplo

In [13]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
    hidden_size=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5930850


## Assert da Perplexidade


In [15]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """
    # Escreva seu código aqui.
    
    crossentropy =  nn.functional.cross_entropy(logits,target)
    
    return crossentropy.exp()


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

my perplexity:              30213
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [16]:
class DataManager(Thread):
    def __init__(self, data_loader, buffer_size=3, device=torch.device("cpu"), data_type=torch.float32):
        """
This manager intends to load a PyTorch dataloader like from disk into memory,
reducing the acess time. It does not easily overflow memory, because we set a
buffer size limiting how many samples will be loaded at once. Everytime a sample
is consumed by the calling thread, another one is replaced in the
buffer (unless we reach the end of dataloader).

A manger may be called exactly like a dataloader, an it's based in an internal
thread that loads samples into memory in parallel. This is specially useful
when you are training in GPU and processor is almost idle.

        :param data_loader: Base dataloader to load in parallel.
        :param buffer_size: How many samples to keep loaded (caution to not overflow RAM). Default: 3.
        :param device: Torch device to put samples in, like torch.device("cpu") (default). It saves time by transfering in parallel.
        :param data_type: Automatically casts tensor type. Default: torch.float32.
        """
        super().__init__()
        self.buffer_queue = Queue(maxsize=buffer_size)
        self.data_loader = data_loader
        self.buffer_size = buffer_size
        self.device = device
        self.data_type = data_type

        self.dataloader_finished = False

    def run(self):
        """
Runs the internal thread that iterates over the dataloader until fulfilling the
buffer or the end of samples.
        """
        for i, (x, y) in enumerate(self.data_loader):
            # Important to set before put in queue to avoid race condition
            # would happen if trying to get() in next() method before setting this flag
            if i >= len(self) - 1:
                self.dataloader_finished = True

            self.buffer_queue.put([x.to(self.data_type).to(self.device),
                                   y.to(self.data_type).to(self.device)])

    def __iter__(self):
        """
Returns an iterable of itself.

        :return: Iterable around this class.
        """
        self.start()
        self.dataloader_finished = False
        return self

    def __next__(self):
        """
Intended to be used as iterator.

        :return: Next iteration element.
        """
        if self.dataloader_finished is True and self.buffer_queue.empty():
            raise StopIteration()

        return self.buffer_queue.get()

    def __len__(self):
        return len(self.data_loader)


In [26]:
from torch.optim.lr_scheduler import ExponentialLR, MultiStepLR
from ptk.utils import DataManager

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

max_examples = 200_000_000
eval_every_steps = 5000
lr = 3e-5


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=4, pin_memory=True)
validation_loader = DataLoader(valid_dataset, batch_size=64, pin_memory=True, num_workers=4,)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

best_validation_loss = 999999999

# Esse gamma estranhamente especifico vai reduzir o LR cerca de mil vezes ao final do treino
scheduler = MultiStepLR(optimizer, milestones=[4, 9], gamma=0.1)

def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in DataManager(train_loader, device=device, buffer_size=4, data_type=None):
        loss = train_step(input.to(device), target.to(device))
        train_losses.append(loss)

        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in DataManager(validation_loader, device=device, buffer_size=4, data_type=None)]))

                # scheduler.step()

                # Checkpoint to best models found.
                if best_validation_loss > valid_ppl:
                    # Update the new best loss.
                    best_validation_loss = valid_ppl
                    model.eval()
                    # torch.save(self, "{:.15f}".format(best_validation_loss) + "_checkpoint.pth")
                    torch.save(model, "best_model.pth")
                    torch.save(model.state_dict(), "best_model_state_dict.pth")

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            print("LR: ",  get_lr(optimizer))
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

        # # Only reduce LR if is not too small
        # if get_lr(optimizer) > 3e-5:
        #     scheduler.step()

# At the end of training, save the final model.
model.eval()
torch.save(model, "last_training_model.pth")

# Load best checkpoint
model = torch.load("best_model.pth")


0 steps; 0 examples so far; train ppl: 29786.61, valid ppl: 30176.77
LR:  3e-05
5000 steps; 160000 examples so far; train ppl: 3428.35, valid ppl: 2056.67
LR:  3e-05
10000 steps; 320000 examples so far; train ppl: 1845.47, valid ppl: 1713.42
LR:  3e-05
15000 steps; 480000 examples so far; train ppl: 1613.49, valid ppl: 1534.81
LR:  3e-05


KeyboardInterrupt: 

## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)